In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import itertools

Importar ```icecream```para debugging

In [2]:
from icecream import ic

Definición de las url para scraping

- ```url```: url de la página principal de freepatent
- ```url_page```: url de la página de busca hasta donde se define la página del paginator de la busqueda
- ```url_search```: parte final url que define la ecuación de busqueda

In [ ]:
url = 'https://www.freepatentsonline.com'
url_page = 'https://www.freepatentsonline.com/result.html?p='
url_search = '&srch=xprtsrch&query_txt=ALL%28health+services%29+AND+%28ALL%28customer+servicies%29+OR+%28omnicanal%29+OR+ALL%28analitics%29%29&uspat=on&usapp=on&eupat=on&jp=on&pct=on&depat=on&date_range=last20&stemming=on&sort=relevance'

Definición de cantidad de páginas en la busqueda

In [3]:
pages = 2

Método que extrae los links de las patentes de la página de busqueda.

- Param: ```pages``` - cantidad de páginas en la busqueda
- Return: ```links```- Lista con los links de las patentes encontradas

In [15]:
def extract_links(pages):
    links = []
    for page in range(1,pages+1):
        r = requests.get(url_page + str(page) + url_search)
        soup = BeautifulSoup(r.content,"lxml")
        
        # se leen todas las filas de tablas
        rows = soup.find_all('tr')
        
        # se quitan los dos primeros y dos últimos valores que no son parte de la tabla de resultados
        rows = rows[2:-2]
        
        # Se copian los links de cada una de las filas
        for row in rows:
            links.append(row.find('a')['href'])
    return links

Método que extrae y transforma la información de las patentes y las guarda en un ```DataFrame```

- Param: ```pages``` - cantidad de páginas en la busqueda
- Return: ```df```- DataFrame con los datos de las patentes organizados

In [12]:
def transform_patents(pages):
    links = extract_links(pages)
    table = []
    for link in links:
        # Tiempo de espera para no sobre cargar la página de requests
        time.sleep(1)
        r = requests.get(url + link)
        soup = BeautifulSoup(r.content,"lxml")
        
        dict = {}
        # Se toman los contenedores que tienen el título y valores de los datos de la patente
        containers = soup.find_all('div', attrs={'class':'disp_doc2'})
        for container in containers:
            title = container.find('div', attrs={'class':'disp_elm_title'})
            value = container.find('div', attrs={'class':'disp_elm_text'})
            
            # Si existen título y contenido se guarda la información sino no se hace nada
            try:
                dict[title.getText().strip().replace(':', '')] = re.sub(' +', ' ', value.getText().strip())
                if 'International Classes' in dict.keys():
                    dict['International Classes'] = dict['International Classes'].replace(';', '\n')
            except:
                continue
        
        # Si hay título vacío se elimina
        if '' in dict.keys():
            del dict['']
        table.append(dict)
    
    # Se crea el DataFrame con los datos relevantes de las patentes
    df = pd.DataFrame(table)
    df = df[['Title','Abstract','Inventors','Application Number','Publication Date','Filing Date','Assignee','International Classes']]
        
    return df

Método que extrae los paises de la columna ```Assignee``` teniendo en cuenta el código ```ISO 2``` de los paises 

- Param: ```df```- DataFrame con los datos de las patentes limpios
- Return: ```df```- DataFrame con nueva columna de paises llamada ```Countries```

In [6]:
def set_country(df):
    # Se leen los datos del archivo de paises y se leen como diccionario con el código iso2 como llave
    countries = pd.read_csv('paises.csv')
    countries.rename(columns=lambda x: x.strip(), inplace=True)
    countries['iso2'] = countries['iso2'].str.strip()
    
    # En la columna Assignee se busca el código del país y se guarda el nombre del país en una nueva columna Country
    country_dict = countries.set_index('iso2').to_dict('index')
    df['Country'] = df['Assignee'].apply(lambda x : '-'.join([v['nombre'] if str(k)+')' in x  else '' for k,v in country_dict.items()]))

    df['Country'] = df['Country'].replace('-+', '\n', regex=True)
    df['Country'] = df['Country'].str.strip()
    
    return df

Método que cuenta la cantidad de veces que aparecen las clases de patente de la columna ```International Classes```

- Param: ```df```- DataFrame con los datos de las patentes
- Return: ```df_classes```- DataFrame con la cantidad de veces que apaceren las clases de patentes

In [7]:
def count_classes(df):
    # Se crea una lista de todas las clases de patente
    classes = [x.split('\n') for x in df['International Classes']]
    classes = list(itertools.chain.from_iterable(classes))
    classes = [s.replace("(IPC1-7):", "") for s in classes]
    classes = [s.strip() for s in classes]
    
    # Se crea un nuevo DataFrame con el número de veces que aparecen las clases
    df_classes = pd.DataFrame(classes, columns=['International Classes'])
    df_classes = df_classes['International Classes'].value_counts().rename_axis('International Classes').reset_index(name='Counts')
    
    return df_classes

Método que crea un excel con la información de pantentes y clases

- Param: ```df```- DataFrame con los datos de las patentes

In [8]:
def save_excel(df):
    df_patents = set_country(df)
    df_classes = count_classes(df)
    
    # Se escriben los DataFrame de patentes y clases en un excel
    with pd.ExcelWriter('Patentes.xlsx') as writer:  
        df_patents.to_excel(writer, sheet_name='Patentes')
        df_classes.to_excel(writer, sheet_name='Cantidad de Clases')

In [13]:
df = transform_patents(page)

In [14]:
save_excel(df)